<a href="https://www.kaggle.com/code/mustafadagteki/sql-analyze-international-debt-statistics?scriptVersionId=154765724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 1. The World Bank's international debt data
<p>It's not that we humans only take debts to manage our necessities. A country may also take debt to manage its economy. For example, infrastructure spending is one costly ingredient required for a country's citizens to lead comfortable lives. <a href="https://www.worldbank.org">The World Bank</a> is the organization that provides debt to countries.</p>
<p>In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. We are going to find the answers to questions like: </p>
<ul>
<li>What is the total amount of debt that is owed by the countries listed in the dataset?</li>
<li>Which country owns the maximum amount of debt and what does that amount look like?</li>
<li>What is the average amount of debt owed by countries across different debt indicators?</li>
</ul>
<p>The first line of code connects us to the <code>international_debt</code> database where the table <code>international_debt</code> is residing. Let's first <code>SELECT</code> <em>all</em> of the columns from the <code>international_debt</code> table. Also, we'll limit the output to the first ten rows to keep the output clean.</p>

In [1]:
# import necessary libraries
import pandas as pd
from pandasql import sqldf

In [2]:
# load data
international_debt = pd.read_csv("/kaggle/input/portfolio-projects-data/international_debt.csv")

In [3]:
# duse pandasql to be able to query pandas data frame as if it is a sql table.

psql = lambda q: sqldf(q, globals())

In [4]:
# check first 10 rows of the table
query1 = """
            SELECT *
            FROM international_debt
            LIMIT 10;
         """
            
result1 = psql(query1)
result1

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1
5,Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845.0
6,Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.3
7,Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820.0
8,Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.9
9,Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.7


## 2. Finding the number of distinct countries
<p>From the first ten rows, we can see the amount of debt owed by <em>Afghanistan</em> in the different debt indicators. But we do not know the number of different countries we have on the table. There are repetitions in the country names because a country is most likely to have debt in more than one debt indicator. </p>
<p>Without a count of unique countries, we will not be able to perform our statistical analyses holistically. In this section, we are going to extract the number of unique countries present in the table. </p>

In [5]:
query2 = """
            SELECT 
                COUNT(DISTINCT country_name) AS total_distinct_countries
            FROM international_debt;
        """
result2 = psql(query2)
result2

,total_distinct_countries
0,124


## 3. Finding out the distinct debt indicators
<p>We can see there are a total of 124 countries present on the table. As we saw in the first section, there is a column called <code>indicator_name</code> that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called <code>indicator_code</code> which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to. </p>

In [6]:
query3 = """
            SELECT 
                DISTINCT indicator_code AS distinct_debt_indicators
            FROM international_debt
            ORDER BY distinct_debt_indicators;
        """

result3 = psql(query3)
result3

,distinct_debt_indicators
0,DT.AMT.BLAT.CD
1,DT.AMT.DLXF.CD
2,DT.AMT.DPNG.CD
3,DT.AMT.MLAT.CD
4,DT.AMT.OFFT.CD
5,DT.AMT.PBND.CD
6,DT.AMT.PCBK.CD
7,DT.AMT.PROP.CD
8,DT.AMT.PRVT.CD
9,DT.DIS.BLAT.CD


## 4. Totaling the amount of debt owed by the countries
<p>As mentioned earlier, the financial debt of a particular country represents its economic state. But if we were to project this on an overall global scale, how will we approach it?</p>
<p>Let's switch gears from the debt indicators now and find out the total amount of debt (in USD) that is owed by the different countries. This will give us a sense of how the overall economy of the entire world is holding up.</p>

In [7]:
query4 = """
            SELECT 
            ROUND((SUM(debt)/1000000), 2) AS total_debt
            FROM international_debt; 
         """

result4 = psql(query4)
result4

,total_debt
0,3079734.49


## 5. Country with the highest debt
<p>"Human beings cannot comprehend very large or very small numbers. It would be useful for us to acknowledge that fact." - <a href="https://en.wikipedia.org/wiki/Daniel_Kahneman">Daniel Kahneman</a>. That is more than <em>3 million <strong>million</strong></em> USD, an amount which is really hard for us to fathom. </p>
<p>Now that we have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. <strong>Note</strong> that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios. We can also find out the category in which the country owns its highest debt. But we will leave that for now. </p>

In [8]:
query5 = """
            SELECT 
                country_name, 
                SUM(debt) AS total_debt
            FROM international_debt
            GROUP BY country_name
            ORDER BY total_debt DESC
            LIMIT 1;
         """
result5 = psql(query5)
result5

,country_name,total_debt
0,China,2.857935e+11


## 6. Average amount of debt across indicators
<p>So, it was <em>China</em>. A more in-depth breakdown of China's debts can be found <a href="https://datatopics.worldbank.org/debt/ids/country/CHN">here</a>. </p>
<p>We now have a brief overview of the dataset and a few of its summary statistics. We already have an idea of the different debt indicators in which the countries owe their debts. We can dig even further to find out on an average how much debt a country owes? This will give us a better sense of the distribution of the amount of debt across different indicators.</p>

In [9]:
query6 = """
            SELECT 
                indicator_code AS debt_indicator,
                indicator_name,
                AVG(debt) AS average_debt
            FROM international_debt
            GROUP BY debt_indicator, indicator_name
            ORDER BY average_debt DESC
            LIMIT 10;
         """
result6 = psql(query6)
result6

,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5.904868e+09
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5.161194e+09
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2.152041e+09
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1.958983e+09
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1.803694e+09
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1.644024e+09
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1.223139e+09
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1.220411e+09
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1.191188e+09
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1.082624e+09


## 7. The highest amount of principal repayments
<p>We can see that the indicator <code>DT.AMT.DLXF.CD</code> tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. More information about this category can be found <a href="https://datacatalog.worldbank.org/principal-repayments-external-debt-long-term-amt-current-us-0">here</a>. </p>
<p>An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. This indicates that the first two indicators might be the most severe categories in which the countries owe their debts.</p>
<p>We can investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (<code>DT.AMT.DLXF.CD</code>). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow us to understand that particular country's economic condition a bit more specifically. </p>

In [10]:
query7 = """
            SELECT 
                country_name, 
                indicator_name
            FROM international_debt
            WHERE debt = (SELECT 
                            MAX(debt)
                        FROM international_debt
                        WHERE indicator_code = 'DT.AMT.DLXF.CD');
         """
result7 = psql(query7)
result7

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."


## 8. The most common debt indicator
<p>China has the highest amount of debt in the long-term debt (<code>DT.AMT.DLXF.CD</code>) category. This is verified by <a href="https://data.worldbank.org/indicator/DT.AMT.DLXF.CD?end=2018&most_recent_value_desc=true">The World Bank</a>. It is often a good idea to verify our analyses like this since it validates that our investigations are correct. </p>
<p>We saw that long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out. </p>

In [11]:
query8 = """
            SELECT
                indicator_code,
                COUNT(*) AS indicator_count
            FROM international_debt
            GROUP BY indicator_code
            ORDER BY indicator_count, indicator_code
            LIMIT 20;
         """
result8 = psql(query8)
result8

,indicator_code,indicator_count
0,DT.DIS.PROP.CD,19
1,DT.DIS.PCBK.CD,51
2,DT.DIS.PRVT.CD,53
3,DT.AMT.PROP.CD,54
4,DT.INT.PROP.CD,54
5,DT.AMT.PBND.CD,69
6,DT.INT.PBND.CD,69
7,DT.AMT.DPNG.CD,79
8,DT.INT.DPNG.CD,79
9,DT.AMT.PCBK.CD,84


## 9. Other viable debt issues and conclusion
<p>There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator <code>DT.AMT.DLXF.CD</code> is also there in the list. So, this gives us a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, but just a part of the story.</p>
<p>Let's change tracks from <code>debt_indicator</code>s now and focus on the amount of debt again. Let's find out the maximum amount of debt that each country has. With this, we will be in a position to identify the other plausible economic issues a country might be going through.</p>
<p>In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.</p>

In [12]:
query9 = """
            SELECT
                country_name,
                MAX(debt) AS maximum_debt
            FROM international_debt
            GROUP BY country_name
            ORDER BY maximum_debt
            LIMIT 10;
         """
result9 = psql(query9)
result9

,country_name,maximum_debt
0,Tonga,10369910.6
1,Sao Tome and Principe,10636848.5
2,Comoros,13460035.4
3,Dominica,21394158.2
4,Samoa,21715408.4
5,Timor-Leste,21799986.4
6,Turkmenistan,29132060.9
7,Solomon Islands,30749703.9
8,Eritrea,31110077.8
9,Somalia,32985034.3
